In [28]:
import pandas as pd
df = pd.read_csv('csv/이자보상배율_금융업제외.csv', index_col=0)
# df = df.drop(columns=['레이블'])
df

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율
0,A005930,삼성전자,2011,1.564429e+10,644133000.0,24.287361
1,A005930,삼성전자,2012,2.904934e+10,599006000.0,48.495905
2,A005930,삼성전자,2013,3.678501e+10,509658000.0,72.175877
3,A005930,삼성전자,2014,2.502507e+10,592940000.0,42.205065
4,A005930,삼성전자,2015,2.641344e+10,776511000.0,34.015541
...,...,...,...,...,...,...
30125,A950010,평산차업 KDR,2012,-2.225668e+06,1648469.0,-1.350142
30126,A950010,평산차업 KDR,2013,1.140593e+06,3146190.0,0.362532
30127,A950010,평산차업 KDR,2014,1.700086e+07,9879818.0,1.720767
30128,A950070,중국고섬,2011,-8.693164e+06,1150594.0,-7.555371


In [29]:
def label_company_group(group):
    group['레이블'] = '-'  # 모든 레코드에 대한 초기 레이블 설정
    in_distress = False  # 부실징후 상태를 추적하는 플래그
    to_set_distress = False  # 다음 해에 부실징후를 설정하기 위한 플래그

    for i in range(len(group)):
        current_icsr = group.iloc[i]['이자보상배율']  # 현재 연도의 이자보상배율

        # 0 미만인 경우 즉시 부실징후로 레이블링
        if current_icsr < 0:
            group.loc[group.index[i], '레이블'] = '부실징후'
            in_distress = True

        # 다음 해에 부실징후로 설정
        if to_set_distress:
            group.loc[group.index[i], '레이블'] = '부실징후'
            in_distress = True
            to_set_distress = False

        # 연속해서 0 이상 1 미만인 경우 다음 해에 부실징후 시작
        if i < len(group) - 1:
            next_icsr = group.iloc[i + 1]['이자보상배율']
            if 0 <= current_icsr < 1 and 0 <= next_icsr < 1:
                to_set_distress = True

        # 회복 조건: 이자보상배율이 1 이상인 연속 두 해 검사
        if i > 0:
            prev_icsr = group.iloc[i - 1]['이자보상배율']
            if prev_icsr >= 1 and current_icsr >= 1:
                if group.iloc[i - 1]['레이블'] == '부실징후':  # 이전 해가 부실징후 상태였다면
                    group.loc[group.index[i - 1], '레이블'] = '회복'
                    group.loc[group.index[i], '레이블'] = '-'
                    in_distress = False  # 회복되면 부실징후 상태 해제

        # 부실징후 상태가 지속되면 레이블 유지
        if in_distress and group.loc[group.index[i], '레이블'] == '-':
            group.loc[group.index[i], '레이블'] = '부실징후'

    # 마지막 해에 대한 회복 검사 (인덱스 범위 초과 방지)
    if len(group) > 1:
        current_icsr = group.iloc[-1]['이자보상배율']
        prev_icsr = group.iloc[-2]['이자보상배율']
        if prev_icsr >= 1 and current_icsr >= 1:
            if group.iloc[-2]['레이블'] == '부실징후':
                group.loc[group.index[-2], '레이블'] = '회복'
                group.loc[group.index[-1], '레이블'] = '-'

    return group

# 회사별로 그룹화하여 레이블링 처리
grouped = df.groupby('Name')
df = pd.concat([label_company_group(group) for name, group in grouped])

In [30]:
df['레이블'].value_counts()

레이블
-       19132
부실징후     9759
회복       1239
Name: count, dtype: int64

In [31]:
def label(label):
    if label == '부실징후':
        return 0
    elif label == '회복':
        return 1
    else :
        return 2


In [32]:
df['label'] = df['레이블'].apply(label)
df

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label
30076,A900010,3노드디지탈,2011,-835891.0,6932080.0,-0.120583,부실징후,0
30077,A900010,3노드디지탈,2012,3471574.0,6501855.0,0.533936,부실징후,0
28624,A038120,AD모터스,2011,-8241498.0,406714.0,-20.263620,부실징후,0
28625,A038120,AD모터스,2012,-4999892.0,103473.0,-48.320741,부실징후,0
28124,A013340,AJS,2011,2281287.0,1647202.0,1.384947,-,2
...,...,...,...,...,...,...,...,...
20186,A238490,힘스,2019,7702123.0,112562.0,68.425605,회복,1
20187,A238490,힘스,2020,24968175.0,114160.0,218.712115,-,2
20188,A238490,힘스,2021,-4371186.0,83511.0,-52.342637,부실징후,0
20189,A238490,힘스,2022,-13275485.0,126361.0,-105.059987,부실징후,0


In [33]:
df.to_csv('csv/rawdata_label.csv', encoding='utf-8-sig')

In [17]:
df['label_shifted'] = df.groupby('Name')['label'].shift(-1)
df

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,label_shifted
30076,A900010,3노드디지탈,2011,-835891.0,6932080.0,-0.120583,부실징후,0,0.0
30077,A900010,3노드디지탈,2012,3471574.0,6501855.0,0.533936,부실징후,0,NaN
28624,A038120,AD모터스,2011,-8241498.0,406714.0,-20.263620,부실징후,0,0.0
28625,A038120,AD모터스,2012,-4999892.0,103473.0,-48.320741,부실징후,0,NaN
28124,A013340,AJS,2011,2281287.0,1647202.0,1.384947,-,2,2.0
...,...,...,...,...,...,...,...,...,...
20186,A238490,힘스,2019,7702123.0,112562.0,68.425605,회복,1,2.0
20187,A238490,힘스,2020,24968175.0,114160.0,218.712115,-,2,0.0
20188,A238490,힘스,2021,-4371186.0,83511.0,-52.342637,부실징후,0,0.0
20189,A238490,힘스,2022,-13275485.0,126361.0,-105.059987,부실징후,0,0.0


In [18]:
df['label'].value_counts()

label
2    19132
0     9759
1     1239
Name: count, dtype: int64

In [19]:
df['label_shifted'].value_counts()

label_shifted
2.0    17072
0.0     9149
1.0     1239
Name: count, dtype: int64

In [20]:
df.to_csv('csv/최종_rawdata_레이블.csv', encoding='utf-8-sig')

In [21]:
df2 = df[(df['레이블'] == '부실징후') | (df['레이블'] == '회복')]
df2

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,label_shifted
30076,A900010,3노드디지탈,2011,-835891.0,6932080.0,-0.120583,부실징후,0,0.0
30077,A900010,3노드디지탈,2012,3471574.0,6501855.0,0.533936,부실징후,0,NaN
28624,A038120,AD모터스,2011,-8241498.0,406714.0,-20.263620,부실징후,0,0.0
28625,A038120,AD모터스,2012,-4999892.0,103473.0,-48.320741,부실징후,0,NaN
28126,A013340,AJS,2013,-43973579.0,2438294.0,-18.034568,부실징후,0,0.0
...,...,...,...,...,...,...,...,...,...
20185,A238490,힘스,2018,-1708653.0,468366.0,-3.648115,부실징후,0,1.0
20186,A238490,힘스,2019,7702123.0,112562.0,68.425605,회복,1,2.0
20188,A238490,힘스,2021,-4371186.0,83511.0,-52.342637,부실징후,0,0.0
20189,A238490,힘스,2022,-13275485.0,126361.0,-105.059987,부실징후,0,0.0


In [22]:
df2.to_csv('csv/최종_labeling(null).csv', encoding='utf-8-sig')

In [23]:
df2.dropna(inplace=True)
df2

/var/folders/lj/ms0l79910fz22cg2hchczhw40000gn/T/ipykernel_70404/3565856863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(inplace=True)


,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,label_shifted
30076,A900010,3노드디지탈,2011,-835891.0,6932080.0,-0.120583,부실징후,0,0.0
28624,A038120,AD모터스,2011,-8241498.0,406714.0,-20.263620,부실징후,0,0.0
28126,A013340,AJS,2013,-43973579.0,2438294.0,-18.034568,부실징후,0,0.0
28127,A013340,AJS,2014,-1031978.0,2915115.0,-0.354009,부실징후,0,0.0
28128,A013340,AJS,2015,-809302.0,700890.0,-1.154678,부실징후,0,0.0
...,...,...,...,...,...,...,...,...,...
20183,A238490,힘스,2016,6575532.0,440975.0,14.911349,회복,1,2.0
20185,A238490,힘스,2018,-1708653.0,468366.0,-3.648115,부실징후,0,1.0
20186,A238490,힘스,2019,7702123.0,112562.0,68.425605,회복,1,2.0
20188,A238490,힘스,2021,-4371186.0,83511.0,-52.342637,부실징후,0,0.0


In [24]:
df2 = df2.drop(df2.loc[df2['label_shifted'] == 2].index)
df2

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,label_shifted
30076,A900010,3노드디지탈,2011,-835891.0,6932080.0,-0.120583,부실징후,0,0.0
28624,A038120,AD모터스,2011,-8241498.0,406714.0,-20.263620,부실징후,0,0.0
28126,A013340,AJS,2013,-43973579.0,2438294.0,-18.034568,부실징후,0,0.0
28127,A013340,AJS,2014,-1031978.0,2915115.0,-0.354009,부실징후,0,0.0
28128,A013340,AJS,2015,-809302.0,700890.0,-1.154678,부실징후,0,0.0
...,...,...,...,...,...,...,...,...,...
17952,A037440,희림,2014,-9859339.0,2880879.0,-3.422337,부실징후,0,1.0
20182,A238490,힘스,2015,-1118085.0,250162.0,-4.469444,부실징후,0,1.0
20185,A238490,힘스,2018,-1708653.0,468366.0,-3.648115,부실징후,0,1.0
20188,A238490,힘스,2021,-4371186.0,83511.0,-52.342637,부실징후,0,0.0


In [25]:
df2.to_csv('csv/최종_labelilng.csv', encoding='utf-8-sig')

In [26]:
df2['label'].value_counts()

label
0    8486
Name: count, dtype: int64

In [27]:
df2['label_shifted'].value_counts()

label_shifted
0.0    7247
1.0    1239
Name: count, dtype: int64

In [35]:
df = pd.read_csv('csv/찐최종_rawdata.csv', index_col=0)
df

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,상장폐지해,상장해
0,A019120,경윤하이드로,2011,-3224672.0,405046.0,-7.961249,부실징후,0,2011.0,1992.0
1,A072530,나이스메탈,2011,-3606551.0,2433905.0,-1.481796,부실징후,0,2011.0,2003.0
2,A037540,네스테크,2011,-2289328.0,935340.0,-2.447589,부실징후,0,2011.0,1999.0
3,A031990,대선조선,2011,-73467986.0,14121516.0,-5.202557,부실징후,0,2011.0,1997.0
4,A005350,봉신,2011,-4765803.0,11385227.0,-0.418595,부실징후,0,2011.0,1988.0
...,...,...,...,...,...,...,...,...,...,...
29431,A900270,헝셩그룹,2019,43907646.0,2179149.0,20.148988,-,2,2025.0,2016.0
29432,A900270,헝셩그룹,2020,5002990.0,2056594.0,2.432658,-,2,2025.0,2016.0
29433,A900270,헝셩그룹,2021,2954130.0,2122802.0,1.391618,-,2,2025.0,2016.0
29434,A900270,헝셩그룹,2022,4330880.0,2538791.0,1.705883,-,2,2025.0,2016.0


In [36]:
df['label_shifted'] = df.groupby('Name')['label'].shift(-1)
df

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,상장폐지해,상장해,label_shifted
0,A019120,경윤하이드로,2011,-3224672.0,405046.0,-7.961249,부실징후,0,2011.0,1992.0,NaN
1,A072530,나이스메탈,2011,-3606551.0,2433905.0,-1.481796,부실징후,0,2011.0,2003.0,NaN
2,A037540,네스테크,2011,-2289328.0,935340.0,-2.447589,부실징후,0,2011.0,1999.0,NaN
3,A031990,대선조선,2011,-73467986.0,14121516.0,-5.202557,부실징후,0,2011.0,1997.0,NaN
4,A005350,봉신,2011,-4765803.0,11385227.0,-0.418595,부실징후,0,2011.0,1988.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29431,A900270,헝셩그룹,2019,43907646.0,2179149.0,20.148988,-,2,2025.0,2016.0,2.0
29432,A900270,헝셩그룹,2020,5002990.0,2056594.0,2.432658,-,2,2025.0,2016.0,2.0
29433,A900270,헝셩그룹,2021,2954130.0,2122802.0,1.391618,-,2,2025.0,2016.0,2.0
29434,A900270,헝셩그룹,2022,4330880.0,2538791.0,1.705883,-,2,2025.0,2016.0,0.0


In [40]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,상장폐지해,상장해,label_shifted
0,A038120,AD모터스,2011,-8241498.0,406714.0,-20.263620,부실징후,0,2012.0,2000.0,0.0
1,A071660,SSCP,2011,-360760.0,16968467.0,-0.021261,부실징후,0,2012.0,2005.0,0.0
2,A105070,금강제강,2011,-7421122.0,3801101.0,-1.952361,부실징후,0,2012.0,2010.0,0.0
3,A004550,대우송도개발,2011,-23280151.0,216234000.0,-0.107662,부실징후,0,2012.0,1978.0,0.0
4,A045260,디에이치패션,2011,-4227818.0,539727.0,-7.833253,부실징후,0,2012.0,2001.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
22415,A900270,헝셩그룹,2018,34536636.0,2160490.0,15.985557,-,2,2025.0,2016.0,2.0
22416,A900270,헝셩그룹,2019,43907646.0,2179149.0,20.148988,-,2,2025.0,2016.0,2.0
22417,A900270,헝셩그룹,2020,5002990.0,2056594.0,2.432658,-,2,2025.0,2016.0,2.0
22418,A900270,헝셩그룹,2021,2954130.0,2122802.0,1.391618,-,2,2025.0,2016.0,2.0


In [41]:
df2 = df[(df['레이블'] == '부실징후') | (df['레이블'] == '회복')]
df2

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,상장폐지해,상장해,label_shifted
0,A038120,AD모터스,2011,-8241498.0,406714.0,-20.263620,부실징후,0,2012.0,2000.0,0.0
1,A071660,SSCP,2011,-360760.0,16968467.0,-0.021261,부실징후,0,2012.0,2005.0,0.0
2,A105070,금강제강,2011,-7421122.0,3801101.0,-1.952361,부실징후,0,2012.0,2010.0,0.0
3,A004550,대우송도개발,2011,-23280151.0,216234000.0,-0.107662,부실징후,0,2012.0,1978.0,0.0
4,A045260,디에이치패션,2011,-4227818.0,539727.0,-7.833253,부실징후,0,2012.0,2001.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
22382,A900050,중국원양자원,2014,-102550556.0,722727.0,-141.893905,부실징후,0,2017.0,2009.0,0.0
22383,A900050,중국원양자원,2015,-17280290.0,231539.0,-74.632308,부실징후,0,2017.0,2009.0,0.0
22390,A900040,차이나그레이트,2017,-18713937.0,3794775.0,-4.931501,부실징후,0,2020.0,2009.0,0.0
22411,A950010,평산차업 KDR,2012,-2225668.0,1648469.0,-1.350142,부실징후,0,2015.0,2007.0,0.0


In [43]:
df2 = df2.drop(df2.loc[df2['label_shifted'] == 2].index)
df2

,Symbol,Name,회계년,영업이익(천원),이자비용(비영업)(천원),이자보상배율,레이블,label,상장폐지해,상장해,label_shifted
0,A038120,AD모터스,2011,-8241498.0,406714.0,-20.263620,부실징후,0,2012.0,2000.0,0.0
1,A071660,SSCP,2011,-360760.0,16968467.0,-0.021261,부실징후,0,2012.0,2005.0,0.0
2,A105070,금강제강,2011,-7421122.0,3801101.0,-1.952361,부실징후,0,2012.0,2010.0,0.0
3,A004550,대우송도개발,2011,-23280151.0,216234000.0,-0.107662,부실징후,0,2012.0,1978.0,0.0
4,A045260,디에이치패션,2011,-4227818.0,539727.0,-7.833253,부실징후,0,2012.0,2001.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
22382,A900050,중국원양자원,2014,-102550556.0,722727.0,-141.893905,부실징후,0,2017.0,2009.0,0.0
22383,A900050,중국원양자원,2015,-17280290.0,231539.0,-74.632308,부실징후,0,2017.0,2009.0,0.0
22390,A900040,차이나그레이트,2017,-18713937.0,3794775.0,-4.931501,부실징후,0,2020.0,2009.0,0.0
22411,A950010,평산차업 KDR,2012,-2225668.0,1648469.0,-1.350142,부실징후,0,2015.0,2007.0,0.0


In [44]:
df2['label_shifted'].value_counts()

label_shifted
0.0    6054
1.0     974
Name: count, dtype: int64

In [47]:
df3 = df2[['Symbol','Name','회계년','label_shifted']]
df3

,Symbol,Name,회계년,label_shifted
0,A038120,AD모터스,2011,0.0
1,A071660,SSCP,2011,0.0
2,A105070,금강제강,2011,0.0
3,A004550,대우송도개발,2011,0.0
4,A045260,디에이치패션,2011,0.0
...,...,...,...,...
22382,A900050,중국원양자원,2014,0.0
22383,A900050,중국원양자원,2015,0.0
22390,A900040,차이나그레이트,2017,0.0
22411,A950010,평산차업 KDR,2012,0.0


In [48]:
df2.to_csv('csv/최종_label(완성본).csv', encoding='utf-8-sig')
df3.to_csv('csv/최종_label.csv', encoding='utf-8-sig')

In [49]:
df4 = df2[['Symbol','Name','회계년']]
df4.to_csv('csv/최종_기업리스트.csv', encoding='utf-8-sig')